# My iz SSSR Wall

In [1]:
import urllib.request
import json
import pandas as pd
from token_key import k
import ssl
import re
import random
regex = re.compile("[А-ЯЁЙЁ]{3,}") 

In [2]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

## 0. Функция, читающая url

In [3]:
def readUrl(url, key=k):
    url = url + k
    content = urllib.request.urlopen(url, context=ctx)
    decoded_content = content.read().decode("utf-8")
    data = json.loads(decoded_content)
    return data

## 1. Собираем все id постов со страницы

In [4]:
def post_numbers(owner_id:str, offset_limit:int):
    data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&count=100&v=5.95&access_token=".format(owner_id))
    post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
    offset = 101
    while offset < offset_limit:
        data = readUrl("https://api.vk.com/method/wall.get?owner_id={}&offset={}&count=100&v=5.95&access_token=".format(owner_id, offset))
        new_post_ids = {data["response"]["items"][i]["id"]:data["response"]["items"][i]["text"] for i in range(len(data["response"]["items"]))}
        post_ids.update(new_post_ids)
        offset += 100
    return post_ids

In [10]:
# больше подписчиков
post_ids = post_numbers(owner_id="-158209155", 
                        offset_limit=20002)

# https://vk.com/sssr_vse

In [11]:
len(post_ids)

7563

## 2. Собираем все комментарии с капслоком на русском языке по пост-id

In [12]:
def get_comments(owner_id:str,  
                 post_ids:list,
                offset_limit=1002):

    result_entries = []
    for id_ in post_ids:
        offset = 1
        while offset < offset_limit: 
            data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, str(id_), offset))
            try:
                data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                data_list = data.values.tolist()
                lines = [i[2] for i in data_list]
                offset += 100
                for i, line in enumerate(lines):
                    try:
                        if re.search(regex, line):
                            result_entries.append(data_list[i])
                    except TypeError:
                        continue
                
                comment_ids = [i[3] for i in data_list]
                for com_id in comment_ids:
                    comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                    comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
                    comment_data_list = comment_data.values.tolist()
                    lines2 = [i[2] for i in comment_data_list]
                    for i, line in enumerate(lines2):
                        try:
                            if re.search(regex, line):
                                result_entries.append(comment_data_list[i])
                        except TypeError:
                            pass
                        
                if offset%100 == 1:
                    print(offset, "ids parsed")

            except KeyError:
                continue
                
    return result_entries

In [18]:
#capslock_comments = get_comments(owner_id="-145800770",
#                                 post_ids=post_ids,
#                                 offset_limit=1002)

In [13]:
offset_limit = 1001
owner_id = "-158209155"

result_entries = []
count = 0
count_ids = 0
for id_, text in post_ids.items():
    offset = 1
    while offset < offset_limit: 
        data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&post_id={}&offset={}&count=100&v=6.7&access_token=".format(owner_id, id_, offset))
        try:
            data = pd.DataFrame(data["response"]["items"], columns = ["from_id", "post_id", "text", "id"])
            data_list = data.values.tolist()
            for i, lst in enumerate(data_list):
                data_list[i].append(text)
            lines = [i[2] for i in data_list]
            offset += 100
            for i, line in enumerate(lines):
                try:
                    if re.search(regex, line):
                        result_entries.append(data_list[i])
                        count += 1
                        if count%100 == 0:
                            print(count, "comments with capslock collected")
                except TypeError:
                    continue
                
            comment_ids_and_text = {i[3]:i[2] for i in data_list}
            for com_id, com_text in comment_ids_and_text.items():
                comment_data = readUrl("https://api.vk.com/method/wall.getComments?owner_id={}&comment_id={}&count=100&v=6.7&access_token=".format(owner_id, com_id))
                comment_data = pd.DataFrame(comment_data["response"]["items"], columns = ["from_id", "parents_stack", "text", "id"])
                comment_data_list = comment_data.values.tolist()
                for i, lst in enumerate(comment_data_list):
                    comment_data_list[i].append(com_text)
                lines2 = [i[2] for i in comment_data_list]
                for i, line in enumerate(lines2):
                    try:
                        if re.search(regex, line):
                            result_entries.append(comment_data_list[i])
                            count += 1
                            if count%100 == 0:
                                print(count, "comments with capslock collected")
                    except TypeError:
                        pass
        except KeyError:
            continue
    count_ids += 1
    if count_ids % 50 == 0:
        print(count_ids, "ids proceeded")

100 comments with capslock collected
200 comments with capslock collected
300 comments with capslock collected
50 ids proceeded
400 comments with capslock collected
500 comments with capslock collected
600 comments with capslock collected
700 comments with capslock collected
800 comments with capslock collected
100 ids proceeded
900 comments with capslock collected
1000 comments with capslock collected
1100 comments with capslock collected
1200 comments with capslock collected
1300 comments with capslock collected
1400 comments with capslock collected
150 ids proceeded
1500 comments with capslock collected
1600 comments with capslock collected
1700 comments with capslock collected
1800 comments with capslock collected
1900 comments with capslock collected
200 ids proceeded
2000 comments with capslock collected
2100 comments with capslock collected
2200 comments with capslock collected
2300 comments with capslock collected
250 ids proceeded
2400 comments with capslock collected
2500 com

18600 comments with capslock collected
3000 ids proceeded
18700 comments with capslock collected
18800 comments with capslock collected
18900 comments with capslock collected
19000 comments with capslock collected
3050 ids proceeded
19100 comments with capslock collected
19200 comments with capslock collected
19300 comments with capslock collected
19400 comments with capslock collected
3100 ids proceeded
19500 comments with capslock collected
19600 comments with capslock collected
19700 comments with capslock collected
3150 ids proceeded
19800 comments with capslock collected
19900 comments with capslock collected
20000 comments with capslock collected
20100 comments with capslock collected
3200 ids proceeded
20200 comments with capslock collected
20300 comments with capslock collected
20400 comments with capslock collected
20500 comments with capslock collected
20600 comments with capslock collected
3250 ids proceeded
20700 comments with capslock collected
20800 comments with capslock

KeyboardInterrupt: 

In [14]:
df_res = pd.DataFrame(result_entries, columns = ["from_id", "parent_id", "comment_text", "comment_id", "parent_text"])
df_res.count()

# 2550 ids proceeded
# 31700 comments with capslock collected
# 31800 comments with capslock collected

from_id         34790
parent_id       34790
comment_text    34790
comment_id      34790
parent_text     34592
dtype: int64

In [16]:
df_res.to_csv("t.csv")

In [5]:
df_res = pd.read_csv("t.csv")
df_res.drop(columns = ["Unnamed: 0"], inplace=True)
df_res.head()

,from_id,parent_id,comment_text,comment_id,parent_text
0,38003371.0,1473828,"А вы что хотели, там сталь У12А инструментальн...",1473851,"У кого были такие ✂? Режут до сих пор, даже лу..."
1,371726224.0,1473828,Сделано в СССР,1473877,"У кого были такие ✂? Режут до сих пор, даже лу..."
2,144367593.0,1473828,Знак качества СССР!!!!!,1473880,"У кого были такие ✂? Режут до сих пор, даже лу..."
3,62297519.0,1473828,"Так то не Китай, а наше изделие сделано по ГОС...",1473893,"У кого были такие ✂? Режут до сих пор, даже лу..."
4,38003371.0,[1473881],"[id238840086|Илья], да ну! Ну тогда ответь по ...",1473902,"Чушь, современные в 100 раз лучше режут"


In [6]:
len(df_res)

34790

## 4. Извлекаем инфу о каждом комментаторе и добавляем в датафрейм. 

In [7]:
def chunks(list_of_values, chunk_size=400):
    return [list_of_values[i:i + chunk_size] for i in range(0, len(list_of_values), chunk_size)]

In [8]:
def update_df_with_user_info(df):
    
    df["first_name"] = None 
    df["last_name"] = None 
    df["sex"] = None
    df["bdate"] = None 
    df["city"] = None
    
    df.drop_duplicates(subset=["from_id"], keep='first', inplace=True)
    df["from_id"] = df["from_id"].apply(lambda x: int(x))
    from_ids = [str(df.from_id[i]) for i in df.index]
    df = df.set_index("from_id")
        
    all_data = []
    for ids in chunks(from_ids):
        k = ",".join(ids)
        data = readUrl("https://api.vk.com/method/users.get?user_ids={}&fields=bdate,sex,city&v=6.7&access_token=".format(str(k)))
        all_data += data["response"]
    
    print(all_data[0])
    
    for el in all_data:
        indx = el["id"]
        try:
            el["city"] = el["city"]["title"]
        except KeyError:
            continue
        df.loc[indx, "first_name"] = el.get("first_name", None)
        df.loc[indx, "last_name"] = el.get("last_name", None)
        df.loc[indx, "sex"] = el.get("sex", None)
        df.loc[indx,"bdate"] = el.get("bdate", None)
        df.loc[indx,"city"] = el.get("city", None)
        
    return df

In [9]:
df_res = update_df_with_user_info(df_res)

{'id': 38003371, 'first_name': 'Mikhail', 'last_name': 'Kubanov', 'is_closed': False, 'can_access_closed': True, 'sex': 2, 'bdate': '22.4', 'city': {'id': 2040, 'title': 'Chapaevsk'}}


In [10]:
df_res.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
278851709,527873,"С праздником мужики! В/ч 3809 ММГ 515 ПОГООН, ...",529624,С Днем Пограничника! 🎉🎉🎉,Alexey,Zhgun,2,31.7.1979,Kansk
39708002,527575,"Норм, на базе ГЛОНАСС сделан?",527704,Помните как ходили в походы ?,Sergey,Baluev,2,None,Uglich
12249619,527575,"У него СПД на основе радия226 и фон его, как п...",527728,Помните как ходили в походы ?,None,None,None,None,None
161438449,527575,Компас с СПД😌 радиоактивная милотаа...,527768,Помните как ходили в походы ?,Petr,Nikitin,2,25.1.1987,Reutov
2201258,527575,"С СПД компас, опасный.",529167,Помните как ходили в походы ?,Roman,Zavorotny,2,None,Snezhinsk


## 5. Удалим комменты с пустыми полями sex, bdate, city

In [11]:
df_test = df_res.dropna(subset=["sex", "bdate", "city"],how="all") 

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9591 entries, 38003371 to 2201258
Data columns (total 9 columns):
parent_id       9591 non-null object
comment_text    9591 non-null object
comment_id      9591 non-null int64
parent_text     5966 non-null object
first_name      9591 non-null object
last_name       9591 non-null object
sex             9591 non-null object
bdate           7474 non-null object
city            9591 non-null object
dtypes: int64(1), object(8)
memory usage: 449.6+ KB


In [13]:
df_test.sex.value_counts()
# со 100 постов собралось всего 287 комментариев с капслоком 

2    7503
1    2084
0       4
Name: sex, dtype: int64

In [14]:
df_test.city.value_counts()

Moscow                  1096
Saint Petersburg        1076
Perm                     168
Novosibirsk              160
Samara                   158
Nizhny Novgorod          143
Kazan                    136
Yekaterinburg            135
Chelyabinsk              132
Ufa                      119
Krasnoyarsk              100
Voronezh                  98
Krasnodar                 96
Omsk                      92
Rostov-on-Don             92
Minsk                     79
Kaliningrad               73
Izhevsk                   72
Volgograd                 72
Vologda                   70
Yaroslavl                 68
Cherepovets               65
Saratov                   65
Arkhangelsk               61
Tyumen                    60
Murmansk                  59
Kirov                     56
Sevastopol                56
Tolyatti                  52
Ryazan                    51
                        ... 
México                     1
Nadvoitsy (derevnya)       1
Siversky                   1
Puerto Cortés 

In [15]:
df_test.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city
from_id,,,,,,,,,
38003371,1473828,"А вы что хотели, там сталь У12А инструментальн...",1473851,"У кого были такие ✂? Режут до сих пор, даже лу...",Mikhail,Kubanov,2,22.4,Chapaevsk
144367593,1473828,Знак качества СССР!!!!!,1473880,"У кого были такие ✂? Режут до сих пор, даже лу...",Lenar,Yarullin,2,22.6,Almetyevsk
62297519,1473828,"Так то не Китай, а наше изделие сделано по ГОС...",1473893,"У кого были такие ✂? Режут до сих пор, даже лу...",Aleksandr,Aleksandrov,2,None,Moscow
355952464,1473727,РАНЬШЕ БЫЛО БОЛЬШЕ ДОВЕРИЯ К ЛЮДЯМ...,1473818,И ведь честно пробивали билетики :),Alyona,Balycheva-Antufyeva,1,18.1,Saint Petersburg
458009932,1473486,Иногда тошнило от запаха подогретой вчерашней ...,1473525,В советской школьной столовой... Вы любили сов...,Albert,Sharipov,2,None,Kazan


In [16]:
df_test = df_test.reset_index()

In [17]:
df_test = df_test.drop_duplicates(subset=["from_id"])
df_test.count()

from_id         9591
parent_id       9591
comment_text    9591
comment_id      9591
parent_text     5966
first_name      9591
last_name       9591
sex             9591
bdate           7474
city            9591
dtype: int64

## Пишем функцию, которая собирает инфу о группах юзера

In [18]:
def update_df_with_subskr_info(df, offset_limit=1001):
    
    df = df.set_index("from_id")
    df["group_info"] = None
    indices = list(df.index)
        
    count = 0
    
    for i in indices: 
        offset = 0
        group_info = []
        while offset < offset_limit:
            subskr = readUrl("https://api.vk.com/method/users.getSubscriptions?user_id={}&extended=1&fields=activity,status,description&count=200&offset={}&v=6.7&access_token=".format(str(i), offset))
            offset += 200
            try:
                for j in subskr["response"]["items"]:
                    group = [j["name"], j["status"], j["description"], j["activity"]]
                    group_info.append(group)
            except KeyError:
                continue
            
        if group_info != []:
            group_info = [" -||- ".join(i) for i in group_info]
            group_info = " -|-|- ".join(group_info)
            df.loc[i, "group_info"] = group_info
        
        count += 1
        if count % 100 == 0:
            print(count, "users processed")
        
    return df

In [19]:
df = update_df_with_subskr_info(df_test)

100 users processed
200 users processed
300 users processed
400 users processed
500 users processed
600 users processed
700 users processed
800 users processed
900 users processed
1000 users processed
1100 users processed
1200 users processed
1300 users processed
1400 users processed
1500 users processed
1600 users processed
1700 users processed
1800 users processed
1900 users processed
2000 users processed
2100 users processed
2200 users processed
2300 users processed
2400 users processed
2500 users processed
2600 users processed
2700 users processed
2800 users processed
2900 users processed
3000 users processed
3100 users processed
3200 users processed
3300 users processed
3400 users processed
3500 users processed
3600 users processed
3700 users processed
3800 users processed
3900 users processed
4000 users processed
4100 users processed
4200 users processed
4300 users processed
4400 users processed
4500 users processed
4600 users processed
4700 users processed
4800 users processed
4

In [27]:
df.tail()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info
from_id,,,,,,,,,,
259335134,6711754,"[id211787099|Антонина], именно. Но, к сожалени...",6712100,,Anastasia,Uryutina,1,3.1.2001,Novouralsk,станция №12 -||- всё будет хорошо. -||- -||- ...
49847737,6711754,"[id68582541|Данил], я считаю, что всех чебурек...",6712471,,Grigory,Savenkov,2,18.6.1987,Luanda,None
217998036,6711754,Нужно любить своих детей. ЛЮБИТЬ.,6712676,,Galina,Yudina,1,2.12,Tolyatti,РАБОЧИЕ АВТОВАЗА -||- ПРОФСОЮЗ ЕДИНСТВО ТОЛЬЯТ...
330060237,6711556,"Дааа. "" У меня есть дырка между ног! Да, все с...",6711952,,Deniel,Semenov,2,None,Baley,None
105574664,6711556,"Как жаль, что раньше это была группа ПРО яжмат...",6712341,,Rita,Gusarova,1,8.6.1997,Saint Petersburg,"Onmyoji ASK -||- -||- Спасибо каждому, кто по..."


In [32]:
df["source"] = ["yazhemati" for i in range(len(df))]

In [33]:
df.head()

,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
from_id,,,,,,,,,,,
6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [34]:
df.reset_index(inplace=True)

In [35]:
df.head()

,from_id,parent_id,comment_text,comment_id,parent_text,first_name,last_name,sex,bdate,city,group_info,source
0,6343530,1.17367e+07,Но это был еще не конец. Я только 31 декабря 1...,11736787,"20 лет назад, 9 августа 1999 года, президент Р...",Yulia,Vershinina,1,24.9,Saint Petersburg,None,lentach
1,135863776,1.17367e+07,Лента сама так решила. Во СЛАВУ ИМПЕРАТОРА!,11736862,"20 лет назад, 9 августа 1999 года, президент Р...",Alexey,Krutikov,2,31.8,Novosibirsk,PidorasBlog -||- паблик чечни -||- Магнит гомо...,lentach
2,10862032,1.17367e+07,Первый срок (и часть второго даже) я одобрял е...,11736889,"20 лет назад, 9 августа 1999 года, президент Р...",Maxim,Gerasimov,2,5.8,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach
3,29670766,[11736731],"[id164270524|Костик], я думаю много весёлого б...",11736884,Зря боря подарил стране плешивого фашиста .,Alexey,Kovalenko,2,18.6.1998,Saint Petersburg,None,lentach
4,320342320,[11736787],"[id6343530|Юлия], в какой из бывших республик ...",11737428,Но это был еще не конец. Я только 31 декабря 1...,Pavel,Vladimirovich,2,26.7,Novosibirsk,Лентач -||- Пропаганда здравого смысла -||- -...,lentach


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8741 entries, 0 to 8740
Data columns (total 12 columns):
from_id         8741 non-null int64
parent_id       8741 non-null object
comment_text    8741 non-null object
comment_id      8741 non-null int64
parent_text     8542 non-null object
first_name      8741 non-null object
last_name       8741 non-null object
sex             8741 non-null object
bdate           6875 non-null object
city            8741 non-null object
group_info      6617 non-null object
source          8741 non-null object
dtypes: int64(2), object(10)
memory usage: 478.1+ KB


## 6. Сохраняем все в файл csv

In [37]:
df.to_csv("capitalization-lentach-wall.csv", index=False)